<a href="https://colab.research.google.com/github/priyank9320/text-recognition/blob/main/custom_OCR_fine_tune_SROIE_with_set_transform_WORKING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# # connect drive
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# install
!pip install -q transformers
!pip install -q datasets jiwer

     |████████████████████████████████| 4.7 MB 30.3 MB/s 
     |████████████████████████████████| 6.6 MB 58.7 MB/s 
     |████████████████████████████████| 120 kB 70.7 MB/s 
     |████████████████████████████████| 365 kB 28.0 MB/s 
     |████████████████████████████████| 212 kB 61.1 MB/s 
     |████████████████████████████████| 115 kB 70.7 MB/s 
     |████████████████████████████████| 127 kB 69.9 MB/s 
     |████████████████████████████████| 1.4 MB 63.3 MB/s 
     |████████████████████████████████| 2.3 MB 60.4 MB/s 
     |████████████████████████████████| 104 kB 59.1 MB/s 


In [4]:
# import
from transformers import TrainingArguments, Trainer, ViTFeatureExtractor, BertTokenizer, VisionEncoderDecoderModel, AutoTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset
from PIL import Image
from datasets import load_dataset, DatasetDict

In [5]:
# variables
# model names
vision_model = "google/vit-large-patch32-384"
language_model = "bert-base-multilingual-cased"

Each element of the dataset should return 2 things:
* `pixel_values`, which serve as input to the model.
* `labels`, which are the `input_ids` of the corresponding text in the image.

We use `TrOCRProcessor` to prepare the data for the model. `TrOCRProcessor` is actually just a wrapper around a `ViTFeatureExtractor` (which can be used to resize + normalize images) and a `RobertaTokenizer` (which can be used to encode and decode text into/from `input_ids`). 

In [6]:
# tokenizer for text
tokenizer =  AutoTokenizer.from_pretrained(language_model)
# feature extractor for image
feature_extractor = ViTFeatureExtractor.from_pretrained(vision_model)

Let's initialize the training and evaluation datasets:

In [ ]:
# download data
original_hf_data_sroie = load_dataset('priyank-m/SROIE_2019_text_recognition')

hf_data_sroie = original_hf_data_sroie
hf_data_sroie

In [8]:
# test image
image = hf_data_sroie['train'][0]['image']
image

In [9]:
# squeeze demo
pix = feature_extractor(image, return_tensors="pt").pixel_values
print(pix.shape)
type(pix.squeeze())
pix.squeeze().shape

torch.Size([1, 3, 384, 384])


torch.Size([3, 384, 384])

In [11]:
# pre-processing function
def prep_function(examples, max_target_length=128):
    pixel_values = feature_extractor(examples['image'], return_tensors="pt").pixel_values
    labels = tokenizer(examples['text'], padding="max_length", max_length=max_target_length).input_ids[0]

    labels = [label if label != tokenizer.pad_token_id else -100 for label in labels]
    # new_label = []
    # for label in labels:
    #     if label!=tokenizer.pad_token_id:
    encoding = {"pixel_values": pixel_values, "labels": torch.tensor([labels])} # squeezeis removing the first dimenion which tells the no. of samples

    # here pixel_values is torch.tensor, but once saved in the dataset it becomes a dict
    return encoding

In [31]:
# on-the-fly preprocessing using set_transform

from datasets import Features, Value, Sequence
hf_data_sroie.set_transform(prep_function, output_all_columns=True)

train_dataset = hf_data_sroie['train']
eval_dataset = hf_data_sroie['test'].shuffle().select(range(100))

In [32]:
train_dataset[0] # checking the format of the data

{'pixel_values': tensor([[[0.9922, 0.9922, 0.9922,  ..., 0.9922, 0.9922, 0.9922],
          [0.9922, 0.9922, 0.9922,  ..., 0.9922, 0.9922, 0.9922],
          [0.9922, 0.9922, 0.9922,  ..., 0.9922, 0.9922, 0.9922],
          ...,
          [0.8902, 0.8902, 0.8902,  ..., 1.0000, 1.0000, 1.0000],
          [0.8902, 0.8902, 0.8902,  ..., 1.0000, 1.0000, 1.0000],
          [0.8902, 0.8902, 0.8902,  ..., 1.0000, 1.0000, 1.0000]],
 
         [[0.9843, 0.9843, 0.9843,  ..., 1.0000, 1.0000, 1.0000],
          [0.9843, 0.9843, 0.9843,  ..., 1.0000, 1.0000, 1.0000],
          [0.9843, 0.9843, 0.9843,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [0.9373, 0.9373, 0.9373,  ..., 1.0000, 1.0000, 1.0000],
          [0.9373, 0.9373, 0.9373,  ..., 1.0000, 1.0000, 1.0000],
          [0.9373, 0.9373, 0.9373,  ..., 1.0000, 1.0000, 1.0000]],
 
         [[0.9529, 0.9529, 0.9529,  ..., 1.0000, 1.0000, 1.0000],
          [0.9529, 0.9529, 0.9529,  ..., 1.0000, 1.0000, 1.0000],
          [0.9529, 0.952

In [34]:
# checking the datatype and dimensions of the data
encoding = train_dataset[1]
for k,v in encoding.items():
  print(k, v.shape)

pixel_values torch.Size([3, 384, 384])
labels torch.Size([128])


In [36]:
# checking the decoding works properly
labels = encoding['labels']
labels[labels == -100] = tokenizer.pad_token_id
label_str = tokenizer.decode(labels, skip_special_tokens=True)
print(label_str)

GST @ 6 % INCLUDED IN TOTAL


In [38]:
# length of the datasets
print("Number of training examples:", len(train_dataset))
print("Number of validation examples:", len(eval_dataset))

Number of training examples: 33626
Number of validation examples: 100


In [ ]:
# model
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    vision_model, language_model
)

Importantly, we need to set a couple of attributes, namely:
* the attributes required for creating the `decoder_input_ids` from the `labels` (the model will automatically create the `decoder_input_ids` by shifting the `labels` one position to the right and prepending the `decoder_start_token_id`, as well as replacing ids which are -100 by the pad_token_id)
* the vocabulary size of the model (for the language modeling head on top of the decoder)
* beam-search related parameters which are used when generating text.

In [26]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = tokenizer.vocab_size

# set beam search parameters
model.config.eos_token_id = tokenizer.sep_token_id
model.config.max_length = 64
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

Next, we can define some training hyperparameters by instantiating the `training_args`. Note that there are many more parameters, all of which can be found in the [documentation](https://huggingface.co/transformers/main_classes/trainer.html#seq2seqtrainingarguments). You can for example decide what the batch size is for training/evaluation, whether to use mixed precision training (lower memory), the frequency at which you want to save the model, etc.

We will evaluate the model on the Character Error Rate (CER), which is available in HuggingFace Datasets (see [here](https://huggingface.co/metrics/cer)).

In [27]:
from datasets import load_metric
cer_metric = load_metric("cer")

The compute_metrics function takes an `EvalPrediction` (which is a NamedTuple) as input, and should return a dictionary. The model will return an EvalPrediction at evaluation, which consists of 2 things:
* predictions: the predictions by the model.
* label_ids: the actual ground-truth labels.

In [28]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [29]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

num_train_epochs=50

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    fp16=True, 
    output_dir="/content/drive/MyDrive/model_storage/", 
    logging_steps=10,
    save_steps=1000, # 1 epoch 
    eval_steps=10, # after these many steps/batches the eval runs and prints the loss
    num_train_epochs=num_train_epochs,
    learning_rate=5e-6,
    remove_unused_columns=False # OMG!! WOW !! this was creating all the issues, this line was required because seq2seqtrainer was just not expecting the on-the-fly column names and it was just removing the samples with only image and text columns so the "Number of samples" showed up as 0 while training
    # refer this link where it was advised to use remove_unused_columns=False : https://github.com/huggingface/datasets/issues/1867
)

Let's train! We also provide the `default_data_collator` to the Trainer, which is used to batch together examples.

Note that evaluation takes quite a long time, as we're using beam search for decoding, which requires several forward passes for a given example.

In [40]:
from transformers import default_data_collator

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=feature_extractor,
    #tokenizer=prep_function,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator
)
trainer.train()

Using cuda_amp half precision backend
***** Running training *****
  Num examples = 33626
  Num Epochs = 200
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 3362600


Step,Training Loss,Validation Loss


RuntimeError: ignored

## Inference

Note that after training, you can easily load the model using the .`from_pretrained(output_dir)` method.

For inference on new images, I refer to my inference notebook, that can also be found in my [Transformers Tutorials repository](https://github.com/NielsRogge/Transformers-Tutorials) on Github.

In [ ]:
pix = torch.tensor(train_dataset['pixel_values'][0])
pixel_values = torch.unsqueeze(input=pix, dim=0)
pixel_values.shape

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pixel_values = pixel_values.to(device)

In [ ]:
train_dataset['text'][0]

In [ ]:
# autoregressively generate caption (uses greedy decoding by default)
generated_ids = model.generate(pixel_values, max_new_tokens=50)
generated_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)

## training works for one sample

## - set_transform issues, somehow it changes the data type of output, this can be our next step
important links to explore for above problem:
- https://github.com/huggingface/datasets/blob/401d4c4f9b9594cb6527c599c0e7a72ce1a0ea49/src/datasets/arrow_dataset.py#L1956
- https://huggingface.co/docs/datasets/v2.4.0/en/package_reference/main_classes#datasets.Dataset.set_transform
- maybe try reset_format

## - but we don't reach 100% accuracy even for one sample, probably we need to implement lr decay, this should be our second step, that is achieve 100% accuracy on a single sample

In [ ]:
tokenizer.batch_decode([21934], skip_special_tokens=True)[0]